In [1]:
import torch
from transformers import AutoTokenizer
from model import GPT
from utils import load_config

/home/zhaox/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
cfg_param = '28M'
cfg = load_config(f"configs/config-{cfg_param}.json")
model = GPT(cfg)

checkpoint_path = "models/model_28M_1020_211110.pt.tar"
checkpoint = torch.load(checkpoint_path, map_location='cpu')

number of parameters: 152.24M


/tmp/ipykernel_104255/4113138004.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location='cpu')


In [7]:
state_dict = checkpoint['state_dict']
new_state_dict = {k[7:] if k.startswith('module.') else k: v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125m')
tokenizer.pad_token = tokenizer.eos_token

prompts = [
    "Which function sets the color used for the background of the window in Processing?",
    "What function do you use to draw a line between two points in Processing?",
    "How do you disable the outline (stroke) of shapes in Processing?",
    "Which function allows you to fill shapes with a color in Processing?",
    "What does the ellipse() function do in Processing?",
    "How do you set the frame rate of a sketch in Processing?",
    "What is the purpose of the draw() function in Processing?",
    "How can you load an image file in Processing?",
    "Which function do you use to display an image on the screen in Processing?",
    "What is the use of the mouseX and mouseY variables in Processing?"
]

inputs = tokenizer(prompts, return_tensors='pt', padding=True).to(device)

In [9]:
def generate_batch_text(model, input_ids, attention_mask, max_length=50, temperature=1.0):
    model.eval()
    generated = input_ids
    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(generated)
            logits = outputs[0]
            next_token_logits = logits[:, -1, :] / temperature
            next_token_probs = torch.softmax(next_token_logits, dim=-1)
            next_token_ids = torch.multinomial(next_token_probs, num_samples=1)
            generated = torch.cat([generated, next_token_ids], dim=-1)
    return generated

output_ids = generate_batch_text(
    model,
    inputs['input_ids'],
    inputs['attention_mask'],
    max_length=30,
    temperature=1.0
)

output_texts = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

for prompt, output in zip(prompts, output_texts):
    print(f"Prompt: {prompt}\nResponse: {output[len(prompt):].strip()}\n{'-'*80}")

Prompt: Which function sets the color used for the background of the window in Processing?
Response: ang Center Experience worldsORation SymptomsGene limbioxid enter emotions pig responseal area professionals, project Display floor Cleveland electricity betaing used purposes of Review.
--------------------------------------------------------------------------------
Prompt: What function do you use to draw a line between two points in Processing?
Response: Yourronesep transported dose Injuryana forum boot began adsite tenant-Beheralate Digital enforcementek Methods times Chem therapy detection appearances D basiciateond
--------------------------------------------------------------------------------
Prompt: How do you disable the outline (stroke) of shapes in Processing?
Response: iallyanceclassion weld deviceance: Cardist Cosmeticage tasting Neural musclesilit Motoralide entering efficacy interactive horses Articlelsets textbooks-month issues
-------------------------------------------

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model = AutoModelForCausalLM.from_pretrained('roneneldan/TinyStories-33M')
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
prompt = "What color do you get when you mix red and blue "
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate completion
output = model.generate(input_ids, max_length = 1000, num_beams=1)

# Decode the completion
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated text
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


What color do you get when you mix red and blue?"

The little girl smiled and said, "I get it now! I get it when I mix red and blue."

The man smiled and said, "That's right! You get the colors you want!"

The little girl was so happy that she ran off to get her colors. She was so excited to mix them together. She mixed red and blue and yellow and green and yellow and purple and orange and purple and orange. She mixed them all together and made a beautiful rainbow.

The man watched her and said, "You are so creative! You are a very good maker!"

The little girl smiled and said, "Thank you! I love to mix colors!"



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model = AutoModelForCausalLM.from_pretrained('roneneldan/TinyStories-33M')
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
prompt = "What color do you get when you mix red and blue "
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate completion
output = model.generate(input_ids, max_length = 1000, num_beams=1)

# Decode the completion
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated text
print(output_text)

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m')
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")

# Input prompt
prompt = "what color do you like? "
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Configure generation parameters
generation_config = GenerationConfig(
    max_length=50,              # Limit the number of tokens
    num_beams=5,                # Beam search for better quality
    temperature=0.8,            # Increase randomness slightly
    top_p=0.85,                 # Reduce top-p to control diversity
    repetition_penalty=1.5,     # Stronger penalty for repetition
    no_repeat_ngram_size=2,     # Prevent repetition of 2-grams
    do_sample=True              # Enable sampling to add variety
)

# Generate the output
output = model.generate(input_ids, generation_config=generation_config)

# Decode the generated text
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the result
print(output_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


what color do you like? 
I don't know what color are you looking for, but I would like to know if there is a way to change the color of the image.
Thanks in advance!

A:

  


In [29]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model = AutoModelForCausalLM.from_pretrained('roneneldan/TinyStories-33M')
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
prompt = "What color do you like? "
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate completion
output = model.generate(input_ids, max_length = 1000, num_beams=1)

# Decode the completion
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated text
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What color do you like? 

The little girl smiled and said, "I like purple. It's my favorite color." 

The man smiled back and said, "I like purple too. Do you want to play a game?" 

The little girl nodded and they started to play. They played for a long time until the little girl had to go home. 

The man said goodbye and the little girl waved as he walked away. She was happy to have made a new friend.

